In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'},
 'stage2': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/stage2_data.pkl'}}

In [2]:
model_name = "MLP_1024_baseline_2class"

In [3]:
import pandas as pd
import numpy as np
from joblib import dump, load

np.random.seed(seed=0)

import torch
torch.manual_seed(0)

In [4]:
train_data = pd.read_pickle(settings["train"]["pickle_path"])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"])

train_data = pd.concat([train_data,valid_data],axis = 0)

test_data = pd.read_pickle(settings["test"]["pickle_path"])

In [5]:
columns = ['A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'topic_A', 'topic_B', 
       'A_vector_bert_1024_mean', 'B_vector_bert_1024_mean',
       'pron_vector_bert_1024_mean', 'product_vector_A_bert_1024',
       'product_vector_B_bert_1024']

In [6]:
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = np.array(train_data.label)


X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test = np.array(test_data.label)

In [7]:
from sklearn.model_selection import KFold 
from mlp2 import MLP
from sklearn.metrics import log_loss

score = []
pred_mlp_tr = np.zeros((X_train.shape[0],2))

kf = KFold(n_splits=5) 
for train_index, test_index in kf.split(X_train):
    X_tr, X_ts = X_train[train_index], X_train[test_index]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    X_tr = X_tr[y_tr!=2]
    y_tr = y_tr[y_tr!=2]
    mlp = MLP(X_train.shape[1],hidden = 512,l2 = (0.05,0.15),dropout_rate = 0.80).fit(X_tr, y_tr)
    pred_ts = mlp.predict_proba(X_ts)
    s = log_loss(y_ts[y_ts!=2],pred_ts[y_ts!=2][:,[0,1]])
    score.append(s)
    pred_mlp_tr[test_index] = pred_ts

    

[MLP][MLP][MLP][MLP][MLP]

In [8]:
score

[0.380240335280703,
 0.4102233654964948,
 0.323359286881522,
 0.39710181380982157,
 0.35494130549501757]

In [9]:
np.array(score).mean()

0.3731732213927118

In [10]:
mlp = MLP(X_train.shape[1],hidden = 512,l2 = (0.05,0.15),dropout_rate = 0.80).fit(X_train[y_train!=2], y_train[y_train!=2])

pred_mlp = mlp.predict_proba(X_test)

dump((pred_mlp_tr,pred_mlp), "../ensemble/"+ model_name +'+basic.joblib') 

[MLP]

['../ensemble/MLP_1024_baseline_2class+basic.joblib']

In [11]:
log_loss(y_test[y_test!=2],pred_mlp[y_test!=2][:,[0,1]])

0.3224325441206151

In [12]:
X_train = np.concatenate([X_train,X_test],axis = 0)
y_train = np.concatenate([y_train,y_test],axis = 0)

mlp = MLP(X_train.shape[1],hidden = 512,l2 = (0.05,0.15),dropout_rate = 0.80).fit(X_train[y_train!=2], y_train[y_train!=2])

[MLP]

In [13]:
dump(mlp, model_name +'.joblib') 

['MLP_1024_baseline_2class.joblib']

---------------------------------------------------------------

In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'},
 'stage2': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/stage2_data.pkl'}}

In [2]:
model_name = "MLP_1024_baseline_2class"
import joblib
import pandas as pd
import numpy as np

mlp = joblib.load(model_name +'.joblib')

stage2_data = pd.read_pickle(settings["stage2"]["pickle_path"])

columns = ['A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'topic_A', 'topic_B', 
       'A_vector_bert_1024_mean', 'B_vector_bert_1024_mean',
       'pron_vector_bert_1024_mean', 'product_vector_A_bert_1024',
       'product_vector_B_bert_1024']

X_test = np.concatenate([np.array(list(stage2_data[col])).reshape(stage2_data.shape[0],-1) for col in columns],axis = 1)


pred_mlp= mlp.predict_proba(X_test)

joblib.dump(pred_mlp, "../ensemble/"+ model_name +'+stage2.joblib')

['../ensemble/MLP_1024_baseline_2class+stage2.joblib']

## Viewed by Xingce BAO @4.10